In [1]:
import numpy as np
import re
import string
import pandas as pd
from gensim.models import Doc2Vec
from gensim.models.doc2vec import LabeledSentence
from gensim import utils
from nltk.corpus import stopwords
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from collections import Counter
import os
import getEmbeddings2
import matplotlib.pyplot as plt
import scikitplot.metrics as skplt


Using TensorFlow backend.


In [2]:
def textClean(text):
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = text.lower().split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return (text)


def cleanup(text):
    text = textClean(text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    return text



In [3]:
new_test = "A sequential model which is a linear stack of layers is used. The first layer is an LSTM layer with 300 memory units and it returns sequences. This is done to ensure that the next LSTM layer receives sequences and not just randomly scattered data. A dropout layer is applied after each LSTM layer to avoid overfitting of the model. Finally, we have the last layer as a fully connected layer with a ‘softmax’ activation and neurons equal to the number of unique characters, because we need to output one hot encoded result."

In [4]:
new_test

'A sequential model which is a linear stack of layers is used. The first layer is an LSTM layer with 300 memory units and it returns sequences. This is done to ensure that the next LSTM layer receives sequences and not just randomly scattered data. A dropout layer is applied after each LSTM layer to avoid overfitting of the model. Finally, we have the last layer as a fully connected layer with a ‘softmax’ activation and neurons equal to the number of unique characters, because we need to output one hot encoded result.'

In [5]:
new_test = cleanup(new_test)

In [6]:
new_test

'sequential model linear stack layers used first layer lstm layer 300 memory units returns sequences done ensure next lstm layer receives sequences randomly scattered data dropout layer applied lstm layer avoid overfitting model finally last layer fully connected layer softmax activation neurons equal number unique characters need output one hot encoded result'

In [8]:
cnt = Counter()
x_train = []
x_train.append(new_test.split())
for word in x_train[-1]:
    cnt[word] += 1

In [9]:
x_train

[['sequential',
  'model',
  'linear',
  'stack',
  'layers',
  'used',
  'first',
  'layer',
  'lstm',
  'layer',
  '300',
  'memory',
  'units',
  'returns',
  'sequences',
  'done',
  'ensure',
  'next',
  'lstm',
  'layer',
  'receives',
  'sequences',
  'randomly',
  'scattered',
  'data',
  'dropout',
  'layer',
  'applied',
  'lstm',
  'layer',
  'avoid',
  'overfitting',
  'model',
  'finally',
  'last',
  'layer',
  'fully',
  'connected',
  'layer',
  'softmax',
  'activation',
  'neurons',
  'equal',
  'number',
  'unique',
  'characters',
  'need',
  'output',
  'one',
  'hot',
  'encoded',
  'result']]

In [10]:
cnt

Counter({'sequential': 1,
         'model': 2,
         'linear': 1,
         'stack': 1,
         'layers': 1,
         'used': 1,
         'first': 1,
         'layer': 7,
         'lstm': 3,
         '300': 1,
         'memory': 1,
         'units': 1,
         'returns': 1,
         'sequences': 2,
         'done': 1,
         'ensure': 1,
         'next': 1,
         'receives': 1,
         'randomly': 1,
         'scattered': 1,
         'data': 1,
         'dropout': 1,
         'applied': 1,
         'avoid': 1,
         'overfitting': 1,
         'finally': 1,
         'last': 1,
         'fully': 1,
         'connected': 1,
         'softmax': 1,
         'activation': 1,
         'neurons': 1,
         'equal': 1,
         'number': 1,
         'unique': 1,
         'characters': 1,
         'need': 1,
         'output': 1,
         'one': 1,
         'hot': 1,
         'encoded': 1,
         'result': 1})

In [12]:
top_words = 5000
epoch_num = 5
batch_size = 64


In [13]:
most_common = cnt.most_common(top_words + 1)


In [14]:
most_common

[('layer', 7),
 ('lstm', 3),
 ('model', 2),
 ('sequences', 2),
 ('sequential', 1),
 ('linear', 1),
 ('stack', 1),
 ('layers', 1),
 ('used', 1),
 ('first', 1),
 ('300', 1),
 ('memory', 1),
 ('units', 1),
 ('returns', 1),
 ('done', 1),
 ('ensure', 1),
 ('next', 1),
 ('receives', 1),
 ('randomly', 1),
 ('scattered', 1),
 ('data', 1),
 ('dropout', 1),
 ('applied', 1),
 ('avoid', 1),
 ('overfitting', 1),
 ('finally', 1),
 ('last', 1),
 ('fully', 1),
 ('connected', 1),
 ('softmax', 1),
 ('activation', 1),
 ('neurons', 1),
 ('equal', 1),
 ('number', 1),
 ('unique', 1),
 ('characters', 1),
 ('need', 1),
 ('output', 1),
 ('one', 1),
 ('hot', 1),
 ('encoded', 1),
 ('result', 1)]

In [15]:
word_bank = {}
id_num = 1
for word, freq in most_common:
    word_bank[word] = id_num
    id_num += 1


In [16]:
word_bank

{'layer': 1,
 'lstm': 2,
 'model': 3,
 'sequences': 4,
 'sequential': 5,
 'linear': 6,
 'stack': 7,
 'layers': 8,
 'used': 9,
 'first': 10,
 '300': 11,
 'memory': 12,
 'units': 13,
 'returns': 14,
 'done': 15,
 'ensure': 16,
 'next': 17,
 'receives': 18,
 'randomly': 19,
 'scattered': 20,
 'data': 21,
 'dropout': 22,
 'applied': 23,
 'avoid': 24,
 'overfitting': 25,
 'finally': 26,
 'last': 27,
 'fully': 28,
 'connected': 29,
 'softmax': 30,
 'activation': 31,
 'neurons': 32,
 'equal': 33,
 'number': 34,
 'unique': 35,
 'characters': 36,
 'need': 37,
 'output': 38,
 'one': 39,
 'hot': 40,
 'encoded': 41,
 'result': 42}

In [17]:
for news in x_train:
    i = 0
    while i < len(news):
        if news[i] in word_bank:
            news[i] = word_bank[news[i]]
            i += 1
        else:
            del news[i]

In [18]:
x_train

[[5,
  3,
  6,
  7,
  8,
  9,
  10,
  1,
  2,
  1,
  11,
  12,
  13,
  14,
  4,
  15,
  16,
  17,
  2,
  1,
  18,
  4,
  19,
  20,
  21,
  22,
  1,
  23,
  2,
  1,
  24,
  25,
  3,
  26,
  27,
  1,
  28,
  29,
  1,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42]]

In [19]:
i = 0
while i < len(x_train):
    if len(x_train[i]) > 10:
        i += 1
    else:
        del x_train[i]

In [20]:
x_train

[[5,
  3,
  6,
  7,
  8,
  9,
  10,
  1,
  2,
  1,
  11,
  12,
  13,
  14,
  4,
  15,
  16,
  17,
  2,
  1,
  18,
  4,
  19,
  20,
  21,
  22,
  1,
  23,
  2,
  1,
  24,
  25,
  3,
  26,
  27,
  1,
  28,
  29,
  1,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42]]

In [21]:
max_review_length = 500
X_train = sequence.pad_sequences(x_train, maxlen=max_review_length)

In [22]:
X_train

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 

In [23]:
from keras.models import load_model


In [24]:
%ls

 datasets/                  naive-bayes.py        testString.ipynb
 ExplorData.ipynb           neural-net-keras.py   webapp/
'Fake News Detection.pdf'   neural-net-tf.py      xte.npy
 getEmbeddings2.py          papers/               xte_shuffled.npy
 getEmbeddings.py           Presentation.pptx     xtr.npy
 images/                   'Project Report.pdf'   xtr_shuffled.npy
'Keras Model'/              __pycache__/          yte.npy
 LSTM.ipynb                 README.md             yte_shuffled.npy
'LSTM Model'/               svm.py                ytr.npy
 LSTM.py                    testSomeData.py       ytr_shuffled.npy


In [25]:
model = load_model('./LSTM Model/model.h5')

/home/swd/anaconda3/envs/my_env/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [26]:
print(model.summary())


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160064    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,365
Trainable params: 213,365
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
y_pred = model.predict_classes(X_train)


In [28]:
y_pred

array([[0]], dtype=int32)

In [30]:
y_pred[0][0]

0